In [1]:
import tensorflow as tf

In [3]:
import pandas as pd
df = pd.read_csv('../Cases/human-resources-analytics/HR_comma_sep.csv')
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.10,0.77,6,247,4,0,1,0,sales,low
3,0.92,0.85,5,259,5,0,1,0,sales,low
4,0.89,1.00,5,224,5,0,1,0,sales,low


In [6]:
X, y = df.drop('left', axis=1), df['left']

In [11]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import train_test_split

cte = make_column_transformer(
    (OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore'), make_column_selector(dtype_include=object)),
    (MinMaxScaler(), make_column_selector(dtype_exclude=object))
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=24, stratify=y)

X_train = cte.fit_transform(X_train)
X_test = cte.transform(X_test)


In [14]:
from tensorflow.keras.initializers import GlorotUniform
tf.random.set_seed(42)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(15, activation='relu', input_shape=(18,), kernel_initializer=GlorotUniform(24)),
    tf.keras.layers.Dense(8, activation='relu', kernel_initializer=GlorotUniform(24)),
    tf.keras.layers.Dense(4, activation='sigmoid', kernel_initializer=GlorotUniform(24)),
    tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=GlorotUniform(24))
])

C:\Users\Kartik\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

In [16]:
history = model.fit(X_train, y_train, epochs=300, validation_data=(X_test, y_test))

Epoch 1/300
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7616 - loss: 0.5867 - val_accuracy: 0.7622 - val_loss: 0.5159
Epoch 2/300
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7641 - loss: 0.4902 - val_accuracy: 0.8535 - val_loss: 0.4087
Epoch 3/300
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8667 - loss: 0.3875 - val_accuracy: 0.8746 - val_loss: 0.3257
Epoch 4/300
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8864 - loss: 0.3055 - val_accuracy: 0.9071 - val_loss: 0.2650
Epoch 5/300
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9191 - loss: 0.2509 - val_accuracy: 0.9315 - val_loss: 0.2265
Epoch 6/300
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9338 - loss: 0.2167 - val_accuracy: 0.9418 - val_loss: 0.2009
Epoch 7/300
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9433 - loss: 0.1942 - val_accuracy: 0.9469 - val_loss: 0.1843
Epoch 8/300
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9491 - loss: 0.1791 - val_accu

In [24]:
from sklearn.metrics import accuracy_score, log_loss
y_pred = model.predict(X_test)
log_loss(y_test, y_pred)

141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


0.13065110620289333

In [28]:
import numpy as np
predict_classes = np.where(y_pred > 0.5, 1, 0)
accuracy_score(y_test, predict_classes)

0.9633251833740831